In [43]:
from py2neo import  Graph, Node, Relationship

from datetime import datetime
import os
import uuid


graph = Graph("http://localhost:7474", user="neo4j",password="juce1234")


def create_n4j_csv(json_f):
#Creates csv files for neo4j.
	import json
	import pandas as pd
	with open(json_f, 'r', encoding='UTF-8') as f:
		data = json.load(f)
	o_list = []
	t_list = []
	for d in data:
		parsed_json = json.loads(d)
		a_kw = parsed_json['kw']
		i = 0
		while i < len(a_kw):
			a_title = parsed_json['title']
			for k in a_kw:
				i_list = []
				if a_title == "":
					i_list.append("Nothing")
					t_list.append("Nothing")
				else:
					i_list.append(a_title)
					t_list.append(a_title)
				if k == "":
					i_list.append("Nothing")
				else:
					i_list.append(k)
				o_list.append(i_list)
				# print(i_list)
				i += 1
			# print(o_list)
		k_df = pd.DataFrame(o_list, columns=['title', 'keyword'])
		t_df = pd.DataFrame(t_list, columns=['title'])
	t_df.to_csv('titles.csv', encoding='utf-8')
	k_df.to_csv('keywords.csv', encoding='utf-8')

def create_titles():
	#Creates title nodes
	query = '''
	LOAD CSV WITH HEADERS FROM "file:///titles.csv" AS csvLine1
	WITH csvLine1
	WHERE csvLine1.title_id IS NOT NULL
	MERGE (a_title:Title {title:csvLine1.title,t_id:csvLine1.title_id})
	ON MATCH SET a_title.title = csvLine1.title
	'''
	graph.run(query)

def create_keywords():
	query = '''
	LOAD CSV WITH HEADERS FROM "file:///keywords.csv" AS csvLine3
	WITH csvLine3
	WHERE csvLine3.title_id IS NOT NULL
	MERGE(keyword:Keyword{k_id:csvLine3.keyword_id,Keyword:csvLine3.keyword})
	WITH csvLine3, keyword
	MATCH(title:Title{t_id:csvLine3.title_id})
	MERGE(title)-[r1:HAS_KEYWORDS]->(keyword)
	'''
	graph.run(query)


def get_common_keywords():
	# Find other keywords articles that share keywords have

	query = '''
	MATCH target= (search_query:Title)-[:HAS_KEYWORDS]->(k_id:Keyword)<-[:HAS_KEYWORDS]-(return_query:Title)-[r2:HAS_KEYWORDS]->(rec_k:Keyword)
	WITH rec_k, COLLECT(return_query.name) AS rec_titles, COUNT(*) AS commonTitles
	RETURN rec_k, rec_titles, commonTitles
	ORDER BY commonTitles DESC LIMIT 10;
	'''

	return graph.cypher.execute(query)

#@app.route('/Keyword', methods=['GET'])
def jaccard_kw_similarity():
	#After adding in new articles, run to find which articles are similar to each other. Jaccard score must be at least 0.1
	#create similarity relationship
	query = '''
	MATCH (search_query:Title)-[:HAS_KEYWORDS]->(k_id:Keyword)
	WITH {item:id(search_query), categories: collect(id(k_id))} as userData
	WITH collect(userData) as data
	CALL algo.similarity.jaccard(data, {topK: 1, similarityCutoff: 0.1, write:true})
	YIELD nodes, similarityPairs, write, writeRelationshipType, writeProperty, min, max, mean, stdDev, p25, p50, p75, p90, p95, p99, p999, p100
	RETURN nodes, similarityPairs, write, writeRelationshipType, writeProperty, min, max, mean, p95
	'''

	result = list(graph.run(query))

#@app.route('/Keyword',methods=['GET'])
def jacc_kw_rec(search):
	q_r = []
	query2 = "MATCH (p:Title)-[:SIMILAR]->(other), (other)-[:HAS_KEYWORDS]->(keyword)  " \
			 "where not((p)-[:HAS_KEYWORDS]->(keyword)) " \
			 "and p.title contains \"{search}\"".format(search=search) + \
			 " RETURN keyword LIMIT 3"

	result = graph.run(query2)

	result = result.data()
	for r in result:
		r_j = json.dumps(r)
		loaded_r = json.loads(r_j)
		kw_ex = loaded_r['keyword']['Keyword']
		q_r.append(kw_ex)
	return q_r

#@app.route('/news_g',methods=['POST'])
def create_graph():
        create_n4j_csv()
        create_titles()
        create_keywords()
        jaccard_kw_similarity()

In [44]:
create_titles()

ServiceUnavailable: Cannot send POST request to </db/data/transaction/commit>